In [54]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# 调用 EDGE 浏览器
driver = webdriver.Edge()
url = "https://zhxy.jxyjxy.com:8078/lyuapServer/login?service=http://218.87.96.236:4106/shiro-cas"
driver.get(url)

# 等待页面加载完成
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'form')))

# 使用WebDriver的page_source属性获取页面源代码
page_source = driver.page_source

# 使用BeautifulSoup解析页面源代码
soup = BeautifulSoup(page_source, 'html.parser')

# 直接查找验证码图片元素
img = driver.find_element(By.CSS_SELECTOR, 'form img')
img_src = img.get_attribute('src')
print("Image source:", img_src)


Image source: data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAGQAAAAZCAIAAABIPBwcAAACgUlEQVR42r2ZwU3FMAyGswczICSu%0AiBOIKdgAjkicmYYVOHNiAe6MQqU8WW7s/7ed9D3JQqVtnOSr7dh+7ffvZ5Ov789N+vXl5dyzH6W/%0AaXWXQWYnWpyaDz9wX00rOmTRpe0NUy/COjevpmca5Ngv7L5/4H7Iykubsq/Jv61Kis9XhWWNevH7%0Au0pKRoCGn2JWiVeIEn1Yuxk7HVJeIrgIy2rYwbIfhARgpIi4kh1CdpLRMM0rqQoNby4I4rcuoJu7%0Aay3VrQ5rcF94uHrvko+G7ufvFx/3byL5sNUQo9CC5NFAyvIiZvv4fLtJJsCXYHFL16RcZGglDizX%0AHZC7CR3RkIe1/RVY3AA3RmJc60ekoNFWlgltjccOFI8GWMOjgRcK6tqy7PcYbEpfHJXTDLDCQzkL%0ACxmXcNFP52AhGxdAHNbTyysXFG2QG1pkLTyYuHFpN7R3Qs0Illx3B+w3OyzkqhlYdjGdlBvO7USx%0AZZEcggR4BKvT4SIDhU4G1kROL6RINhNYFgl1KD8eGLmBbA6WDV4TCZcLy4aqMI8NYJGihNy0sJDl%0ADm6oDVZyBVfsDvMxSycQYYWwg8VrERSqSNXiuiEaa2HpaMVhDTtJwpIh+Yw06DqQg9yl46Zd+QCP%0AShN7LOoQluxVoE+byfthUlpSqt9x03dUDyJYKE1BsNAiwxbNcA6W6u02532kNsyfUDpmhdX1cDhm%0Aan5egZIqJ9V1yHQqVprCyYZMWIHNweL1XOaIK8Cq3syHD9Qb4ENKnhjC4ms7tWjCBuPhP5+Qrc5F%0Ag7D7Fla4YU6+gzWsb+Wninx3jR8syS2FnpgsSEJkENYKr1IIC49L3okOC36kP9+D1vIPh++PNG36%0AsNgAAAAASUVORK5C

In [51]:
from PIL import Image
import pytesseract
import sympy as sp
from PIL import Image, ImageOps, ImageFilter,ImageEnhance
import string
import requests
from io import BytesIO
import base64


def preprocess_image(image):
    # 调整大小
    image = image.resize((300, 100))
    # 灰度化
    image = ImageOps.grayscale(image)
    # 二值化
    threshold = 150  # 调整阈值
    image = image.point(lambda p: p > threshold and 255)

    # 去噪
    image = image.filter(ImageFilter.MedianFilter(size=3))

    # 增强对比度
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2.0)

    # 显示预处理后的图像（可选）
    #image.show()

    return image

# Set the path to the Tesseract executable (update this path according to your installation)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def recognize_and_calculate_result(img_src):
    
    base64_image=img_src.replace("data:image/png;base64,","")
    # 将图片数据读取为二进制流
    img_data =  base64.b64decode(base64_image)
    with open("temp_image.png", "wb") as img_file:
        img_file.write(img_data)
    # Open the image
    image = Image.open("temp_image.png")
    image.show()

    # Apply preprocessing (you can customize this based on your needs)
    image = preprocess_image(image)

    # Use Tesseract OCR to extract text from the image
    recognize_text = pytesseract.image_to_string(image, config='--psm 6 -c tessedit_char_whitelist=0123456789+/*=?')
    expression_text = recognize_text.replace("=?","")

    try:
        # Evaluate the expression using sympy
        result = sp.sympify(expression_text)
        return expression_text, result
    except sp.SympifyError:
        return expression_text, "Error: Unable to evaluate the expression"

if __name__ == "__main__":
    # Provide the path to the image containing the math expression

    expression, result = recognize_and_calculate_result(img_src)

    print("Math Expression:", expression)
    print("Result:", result)

OSError: broken data stream when reading image file

In [60]:
from PIL import Image
import base64
from io import BytesIO

# 图像数据
image_data = "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAGQAAAAZCAIAAABIPBwcAAACcUlEQVR42q2ZTUpFMQyFsxVBXIAbEHTiApzKQ3Dkvpy4A1fhIpy6CC8USsjPyUnugyLv1Tav/W5ymubK99fn0X5//rptTzQfTlo7OUYPLk0xY3SP7GndmXri/lcXVmsuGJAtr4RFjlnDhCTtBxhbJCMSepdXF1ZryxZW6bf+aXhYDK8MOullmS+HC+g+fgzraDJQltb2Si8IJY+HBXh1nQt8ncAKAWF1LzszC3giDuexiuPokZmUlk+49HZv5AysazmXUaQGLKwFuv/5/UO3Us4ut5dwQBiJdw83u4XPHCw4I/X2dL8br1zCqAM+vwypjNditGEtO5pauGdNajUcdIw3aVIlsgDWIOANqd3ve7SdhWn/zRzKkNqmDCwyNza7ONB4doxcCA6HkpfhknWaGFwt0w5DCidWTG6Mvy5Y5a7TMMTu5rlgzwKwQn3RsHiFfn18wa2EVWb2rGaBMyjTrNCmD8NsARuWF6wZrFBhNKmSVw8Wdq5Q4P0haJQ+W5mX9pAXk8dkZ5QnhfVa+GQ/NATCMIxuLfNAcTQsoPc8Lz8FHIJpGPLXyNCW9iPDa3B/yjTL8AqNkJq1f2jDuppnlZl06EQ8LHBxMbBWP1AuBhZWq9K5JIuU8hw16m7wnYfleQGZZ+7e/nP3GiTMPRnnFkDgxzXSLH0HyQR/UmF1B7yGsDL/Cu86+DFmsMIzES+JKaXysDwvIZNjUqSzJKNVEeZLToNiCRkx4VVJ+BIPDhlcwDsPi7mBkShJOaZOQ1zoAHpclonPeATDi6fAXHupPIv5SVLpWs7V5XWmNjl7ayfk4nhYg7dkg3cHJKxrvYVc7R+b+fKI84ZbqwAAAABJRU5ErkJggg=="

# 去除数据头部
base64_data = image_data.replace("data:image/png;base64,", "")

# 解码 base64 数据
image_bytes = base64.b64decode(base64_data)

# 读取为图像对象
image = Image.open(BytesIO(image_bytes))

# 显示图像（可选）
image.show()
